In [24]:
#REFS:
#Wildon, Vertices of Specht Modules and Blocks of the Symmetric Group
#Murphy, The Idempotents of the Symmetric Group and Nakayama's Conjecture
#Murphy, A new construction of Young's seminormal representation of the symmetric groups

In [1]:
n=3

In [2]:
SGA = SymmetricGroupAlgebra(QQ,n)

In [3]:
SGA.dft()

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [4]:
SGA_F7 = SymmetricGroupAlgebra(GF(7),3)

In [5]:
SGA_F7.dft()

[1 1 1 1 1 1]
[1 4 6 3 3 4]
[0 6 0 6 1 1]
[0 1 0 6 1 6]
[1 3 1 3 3 3]
[1 6 6 1 1 6]

In [6]:
SGA_F7.specht_module(Partition([1,1,1]))

Specht module of [(0, 0), (1, 0), (2, 0)] over Finite Field of size 7

In [7]:
SGA_F3 = SymmetricGroupAlgebra(GF(3),3)

In [8]:
#one cannot perform the DFT when p | n!
try:
    SGA_F3.dft()
except ZeroDivisionError:
    print("Modular case not handled!")

Modular case not handled!


In [9]:
#Theorem 3.3, "Nakayama's Conjecture", pg. 8, Wildon, Vertices of Specht Modules and Blocks of the Symmetric Group:

#Let p be a prime. The p-blocks of the symmetric group Sn are labelled by pairs (\gamma,w), 
#where \gamma is a p-core and w \in \mathbb{N}_0 is the associated weight, such that |\gamma| + wp = n. 
#Thus S^\lambda lies in the block labelled by (\gamma,w) if and only if \lambda has p-core \gamma and weight w.
#
#---
#
#Theorem 2.1, Murphy, The Idempotents of the Symmetric Group and Nakayama's Conjecture
#
#F_j^i = \sum_{\mu,k | t_k^\mu \in T_j^i} E_k^\mu 
#H^i = \sum_j F_j^i 
#
#{F_j^i} is a complete set of orthogonal idempotents of RS_n, similarly for {\overline{F}_j^i}.
#
#where
#
#E_i^\mu = \prod_{c = -n+1}^{n-1} \prod_{u | a_{ui}^\mu \ne c} \frac{c-L_u}{c-\alpha_{ui}^\mu}
#
#L_u = (1,u) + (2,u) + ... + (u-1,u), a sum of transpositions
#
#T_j^i are tableaux belonging to equivalence classes B_i, where B_i are equivalences classes of
#partitions given by \tilde_p
#
#\labmda \tidle_p \mu iff the classes (j-i) of each node have the same residue mod p
#
#The class of the node occupied by u in t_i^\mu is \alpha_{ui}^\mu where t_1^\mu, ..., t_d^\mu are the standard \mu-tableaux 
#in the ordering in pg. 288 of "A new construction of Young's seminormal representation of the symmetric groups".
#
#The "class" of a node (i,j) in a Young diagram \mu is the difference j-i.
#
#d is the number of "standard \mu-tableaux", given by the Hook-length formula.
#
#---
#
#Theorem 2.8 (Nakayama's Conjecture): {\overline{H}^i} is a complete set of primitive orthogonal central idempotents of \overline{K}S_n,
#and S_{\overline{K}}^\mu, S_{\overline{K}}^\lambda belong to the same block of \overline{K}S_n if and only if \mu \tilde_p \lambda.
#
#---

In [10]:
mu = Partition([2,1]); mu

[2, 1]

In [11]:
list(StandardTableaux(mu))

[[[1, 3], [2]], [[1, 2], [3]]]

In [12]:
#define the element \alpha_{ui}^\mu
#which is the class of u in the i^th standard Tableaux corresponding to mu
#the class of a node is j-i where i is the row and j is the column
def alpha(mu,u,i):
    tab = StandardTableaux(mu)[i]
    for row in range(len(tab)):
        for col in range(len(tab[row])):
            if u == tab[row][col]:
                return col - row

In [13]:
#define the elment L_u = (1,u) + (2,u) + ... + (u-1,u) in K[S_n]
def L(u):
    return sum(SGA(PermutationGroupElement(f"({i},{u})")) for i in range(1,u))

In [15]:
def E(mu,i):
    return prod(prod((c-L(u))/(c-alpha(mu,u,i)) for u in range(1,n+1) if alpha(mu,u,i) != c) for c in range(-n+1,n))

In [23]:
#check idempotency
E(mu,0)*E(mu,0) == E(mu,0)

True

In [18]:
#equivalent to having the same p-core
#look at canonical tableaus (place 1,...,n in top-to-bottom, left-to-right)
#compute p-residues of classes j-i \mod p
def p_equiv(mu_1, mu_2,p):
    return core(mu_1,p) == core(mu_2,p)


In [128]:
#helper function to determine if diagram comes from a partition
def diagram_is_from_partition(diag):
    from_partition = True
    #check if rows start at 0 and have any gaps
    row_labels = {node[0] for node in diag}
    from_partition &= min(row_labels) == 0
    from_partition &= len(row_labels) == max(row_labels) - min(row_labels) + 1
    #for each row, check if columns have any gaps, and start at 0
    for row in row_labels:
        col_labels = {node[1] for node in diag if node[0]==row}
        from_partition &= min(col_labels) == 0
        from_partition &= len(col_labels) == max(col_labels) - min(col_labels) + 1
    return from_partition

In [148]:
#given a diagram which comes from a partition, give the partition
def to_partition(diag):
    assert diagram_is_from_partition(diag)
    return Partition([max({node[1] for node in diag if node[0]==row})+1 for row in {node[0] for node in diag}])

In [149]:
diag=NorthwestDiagrams().from_partition(Partition([2,1]))
to_partition(diag)

[2, 1]

In [176]:
#find the p-core of a partition
#a "rim p-hook" is:
#connected set of p nodes
#in the rim (the node (i+1,j+1) is not in the diagram)
#whose removal leaves a valid young diagram
from sage.combinat.diagram import NorthwestDiagrams
def core(mu,p):
    #get the diagram associated to the partition
    diag = NorthwestDiagrams().from_partition(mu)
    #find the rim of the diagram/partition
    rim = []
    for node in diag:
        if (node[0]+1,node[1]+1) not in diag:
            rim.append(node)
    #find rim p-hooks
    p_hooks = []
    for start in rim:
        p_hook = [start]
        for step in range(p-1):
            current = p_hook[step]
            down = (current[0]+1,current[1])
            left = (current[0],current[1]-1)
            if (not down in rim) and (not left in rim):
                break
            if down in rim:
                p_hook.append(down)
                continue
            if left in rim:
                p_hook.append(left)
                continue
        if len(p_hook) == p:
            p_hooks.append(p_hook)
    #recursive step: if no p-hooks, return original diagram
    if len(p_hooks) == 0:
        return to_partition(diag)
    #determine if removal of candidate p_hook results in a diagram
    valid_p_hook_removed_diags = []
    for p_hook in p_hooks:
        diag_minus_p_hook = Diagram([node for node in diag if node not in p_hook])
        if diagram_is_from_partition(diag_minus_p_hook):
            valid_p_hook_removed_diags.append(diag_minus_p_hook)
    #recursive step: if diagram is unchanged, return, otherwise keep going
    for valid_p_hook_removed_diag in valid_p_hook_removed_diags:
        diag.pp()
        print("-----------")
        valid_p_hook_removed_diag.pp()
        print("###########")
        print(p_hooks)
        partition = to_partition(valid_p_hook_removed_diag)
        return core(partition,p)

In [175]:
core(Partition([6,6,5,1,1]),5)

O O O O O O 
O O O O O O 
O O O O O . 
O . . . . . 
O . . . . . 
-----------
O O O O O 
O O O O . 
O O O . . 
O . . . . 
O . . . . 
###########
[[(0, 5), (1, 5), (1, 4), (2, 4), (2, 3)], [(1, 4), (2, 4), (2, 3), (2, 2), (2, 1)], [(1, 5), (1, 4), (2, 4), (2, 3), (2, 2)], [(2, 2), (2, 1), (2, 0), (3, 0), (4, 0)], [(2, 3), (2, 2), (2, 1), (2, 0), (3, 0)], [(2, 4), (2, 3), (2, 2), (2, 1), (2, 0)]]
O O O O O 
O O O O . 
O O O . . 
O . . . . 
O . . . . 
-----------
O O O 
O O . 
O O . 
O . . 
O . . 
###########
[[(0, 3), (1, 3), (1, 2), (2, 2), (2, 1)], [(0, 4), (0, 3), (1, 3), (1, 2), (2, 2)], [(1, 2), (2, 2), (2, 1), (2, 0), (3, 0)], [(1, 3), (1, 2), (2, 2), (2, 1), (2, 0)], [(2, 2), (2, 1), (2, 0), (3, 0), (4, 0)]]
O O O 
O O . 
O O . 
O . . 
O . . 
-----------
O O O 
O . . 
###########
[[(0, 1), (1, 1), (2, 1), (2, 0), (3, 0)], [(0, 2), (0, 1), (1, 1), (2, 1), (2, 0)], [(1, 1), (2, 1), (2, 0), (3, 0), (4, 0)]]


[3, 1]

In [171]:
if len([])==0:
    print("empty")

empty
